In [ ]:
# clone project and download dependences
!pip install --upgrade --no-cache-dir gdown
from IPython.display import clear_output
!git clone https://github.com/siidev/StyleMapGAN.git
%cd StyleMapGAN/
!pip install ninja
!pip install flask-ngrok
!pip install scikit-image==0.16.2
clear_output()
print('Done!')

Done!


In [ ]:
!gdown --id 1-1IdZ4riYa-lSIw2FT13cOXDkTF84BbW
!unzip afhq.zip

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-1IdZ4riYa-lSIw2FT13cOXDkTF84BbW
To: /content/StyleMapGAN/afhq.zip
100% 824M/824M [00:06<00:00, 128MB/s]
Archive:  afhq.zip
   creating: lmdb-data/afhq/
   creating: lmdb-data/afhq/test/
  inflating: lmdb-data/afhq/test/data.mdb  
  inflating: lmdb-data/afhq/test/lock.mdb  
   creating: lmdb-data/afhq/val/
  inflating: lmdb-data/afhq/val/data.mdb  
  inflating: lmdb-data/afhq/val/lock.mdb  
   creating: lmdb-data/afhq/train/
  inflating: lmdb-data/afhq/train/data.mdb  
  inflating: lmdb-data/afhq/train/lock.mdb  


Read https://github.com/naver-ai/StyleMapGAN/blob/main/metrics/README.md

In [ ]:
# prepare fid
import gdown
import os

url = 'https://drive.google.com/uc?id=1pCr4lNCON7IZcNVdskIDXhFJ3jYuge1w' # inception checkpoint
output = 'metrics/pt_inception-2015-12-05-6726825d.pth'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=1dMHaNkgIy_-cq70f6nqoQsfG9-aOTpSC' # random generated samples as pickle files
output = 'fid_stats.zip'
gdown.download(url, output, quiet=False)
!unzip fid_stats.zip

Downloading...
From: https://drive.google.com/uc?id=1pCr4lNCON7IZcNVdskIDXhFJ3jYuge1w
To: /content/StyleMapGAN/metrics/pt_inception-2015-12-05-6726825d.pth
100%|██████████| 95.6M/95.6M [00:00<00:00, 125MB/s]
Downloading...
From: https://drive.google.com/uc?id=1dMHaNkgIy_-cq70f6nqoQsfG9-aOTpSC
To: /content/StyleMapGAN/fid_stats.zip
100%|██████████| 96.3M/96.3M [00:01<00:00, 57.8MB/s]


Archive:  fid_stats.zip
   creating: metrics/fid_stats/
  inflating: metrics/fid_stats/celeba_hq_stats_256_29000.pkl  
  inflating: metrics/fid_stats/afhq_stats_256_15130.pkl  
  inflating: metrics/fid_stats/vn_celeb_stats_128_22105.pkl  


In [ ]:
# download checkpoint
checkpoint = '500000.pt'
!gdown --id 1jEGb4WT9DmQM5Ec07vTVr5vrL88mMj8v

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1jEGb4WT9DmQM5Ec07vTVr5vrL88mMj8v
To: /content/StyleMapGAN/500000.pt
100% 1.05G/1.05G [00:08<00:00, 127MB/s] 


In [ ]:
# Reconstruction
!python generate.py --ckpt {checkpoint} --mixing_type reconstruction --test_lmdb lmdb-data/afhq/test
!python -m metrics.reconstruction --image_folder_path expr/reconstruction/afhq

Namespace(batch=16, batch_per_gpu=4, channel_multiplier=2, ckpt='408000.pt', d_reg_every=16, dataset='afhq', iter=1400000, lambda_adv_loss=1, lambda_d_loss=1, lambda_indomainGAN_D_loss=1, lambda_indomainGAN_E_loss=1, lambda_perceptual_loss=1, lambda_w_rec_loss=1, lambda_x_rec_loss=1, latent_channel_size=64, latent_spatial_size=8, local_editing_part=None, lr=0.002, lr_mul=0.01, mapping_layer_num=8, mixing_type='reconstruction', ngpus=1, normalize_mode='LayerNorm', num_workers=10, r1=10, save_image_dir='expr', save_network_interval=2000, size=256, small_generator=True, start_iter=396000, test_lmdb='lmdb-data/afhq/test', train_data='lmdb-data/afhq/train', val_data='lmdb-data/afhq/val')
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker cr

In [ ]:
# FID
!python -m metrics.fid --ckpt {checkpoint} --comparative_fid_pkl metrics/fid_stats/afhq_stats_256_15130.pkl --dataset afhq --size 256

/usr/local/lib/python3.7/dist-packages/torchvision/models/inception.py:83: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)
  0% 0/946 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
100% 946/946 [39:56<00:00,  2.53s/it]
extracted 15130 features
fid: 10.427992644068127


In [ ]:
# Local editing
!python generate.py --mixing_type local_editing --ckpt {checkpoint} --test_lmdb lmdb-data/afhq/test
!python -m metrics.local_editing --data_dir expr/local_editing/afhq

Namespace(batch=16, batch_per_gpu=4, channel_multiplier=2, ckpt='500000.pt', d_reg_every=16, dataset='afhq', iter=1400000, lambda_adv_loss=1, lambda_d_loss=1, lambda_indomainGAN_D_loss=1, lambda_indomainGAN_E_loss=1, lambda_perceptual_loss=1, lambda_w_rec_loss=1, lambda_x_rec_loss=1, latent_channel_size=64, latent_spatial_size=8, local_editing_part=None, lr=0.002, lr_mul=0.01, mapping_layer_num=8, mixing_type='local_editing', ngpus=1, normalize_mode='LayerNorm', num_workers=10, r1=10, save_image_dir='expr', save_network_interval=2000, size=256, small_generator=True, start_iter=498000, test_lmdb='lmdb-data/afhq/test', train_data='lmdb-data/afhq/train', val_data='lmdb-data/afhq/val')
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker cre